In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers kagglehub pandas

!pip install -U langchain-community

!pip install faiss-cpu

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

import pandas as pd
from transformers import pipeline
import kagglehub


In [2]:
# Replace with your dataset path
dataset_path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")
print("Dataset downloaded at:", dataset_path)


Using Colab cache for faster access to the 'sentiment-analysis-dataset' dataset.
Dataset downloaded at: /kaggle/input/sentiment-analysis-dataset


In [3]:
import pandas as pd

csv_file = dataset_path + "/train.csv"  # adjust path if needed

# Use encoding 'latin1' or 'ISO-8859-1'
df = pd.read_csv(csv_file, encoding='latin1')

print(df.head())


       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (Km²)  Density (P/Km²)  
0  Afghanistan          38928346         652860.0    

In [4]:
docs = df['text'].tolist()  # replace 'text' with the actual column name


In [5]:


from langchain.text_splitter import CharacterTextSplitter
import pandas as pd
import numpy as np

# Suppose docs is a list or a pandas column
# Example: docs = df['text_column'].tolist()

# Convert all elements to string and remove NaNs
docs = [str(doc) for doc in docs if pd.notnull(doc)]

# Initialize splitter
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Create documents
documents = splitter.create_documents(docs)

print(f"Number of chunks created: {len(documents)}")


Number of chunks created: 27480


In [6]:
# Use HuggingFace embeddings (free)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS vectorstore
db = FAISS.from_documents(documents, embeddings)
retriever = db.as_retriever()

/tmp/ipython-input-1887289669.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

In [7]:
# Use a small open-source model (free to run in Colab CPU)
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-small")

llm = HuggingFacePipeline(pipeline=llm_pipeline)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-2878127765.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [8]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [9]:
query = "What is the sentiment of the text 'I love this product'?"
result = qa(query)

print("Question:", query)
print("Answer:", result['result'])
print("Source Document Sample:", result['source_documents'][0].page_content[:200], "...")


/tmp/ipython-input-3303470553.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(query)


Question: What is the sentiment of the text 'I love this product'?
Answer: positive
Source Document Sample: hey life, i love you!   (translation: i love life!) ...
